In [2]:
import pandas as pd 

In [34]:
df1=pd.read_csv('New_sunday_oberver.csv')
df22=pd.read_csv('dailyft_batch0000-1500.csv')

In [112]:
df1.head()

,Date,Title,Content
0,17 January 2021,Clearing the hiring hurdles,"In an increasingly competitive world, people h..."
1,14 June 2020,Only the fast and nimble will survive - Pathfi...,Now that Sri Lanka has achieved notable succes...
2,14 February 2021,SEC and CSE meet stockbrokers,Officials of the Securities and Exchange Commi...
3,8 December 2019,Shares to be listed and traded in foreign curr...,The Colombo Stock Exchange (CSE) is ready with...
4,14 March 2021,Kala Pola 2021 goes online!,Kala Pola - Sri Lanka’s annual art fair showca...


In [81]:
text1=df1['Content']

In [78]:
df22.rename(columns = {df22.columns[2]:'Content'},inplace = True)

In [80]:
text2=df22['Content']

In [104]:
import numpy as np

In [91]:
df=pd.concat([text1,text2], axis=0,ignore_index=True)

In [83]:
import re 
def remove_links(text):
   
    remove_https = re.sub(r'http\S+', '', text)
    # Remove all the occurrences of text that ends with .com
    remove_com = re.sub(r"\ [A-Za-z]*\.com", " ", remove_https)
    return remove_com

In [84]:
def remove_whitespace(text):
    pattern = re.compile(r'\s+') 
    Without_whitespace = re.sub(pattern, ' ', text)
    # There are some instances where there is no space after '?' & ')', 
    # So I am replacing these with one space so that It will not consider two words as one token.
    
    text = Without_whitespace.replace('?', ' ? ').replace(')', ') ')
    return text

In [85]:
def RSproblem(text):
    text1 = re.sub('Rs.',  '$', text)
    text2=re.sub('Rupees',  '$', text1)
    return text2

In [95]:
T=df[199]

In [96]:
def preprocess(text):
    final_text=remove_whitespace(remove_links(text))
    fin_text=RSproblem(final_text)
    return fin_text

In [97]:
s=preprocess(T)

In [98]:
df = df.dropna()

In [99]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace("?",".")
    text = text.replace("<prd>",".")
    #sentences = text.split("<stop>")
    #sentences = sentences[:-1]
    #sentences = [s.strip() for s in sentences]
    sentences=decimal_split(text)
    return sentences

In [100]:
def decimal_split(string1):
    # replace dots that have numbers around them with "[PROTECTED_DOT]"
    string1_protected = re.sub(r"(\d)\.(\d)", r"\1[PROTECTED_DOT]\2", string1)  
    # now split (and remove empty lines)
    lines_protected = [line + "." for line in string1_protected.split(".") if line]   
    # now re-replace all "[PROTECTED_DOT]"s
    lines = [line.replace("[PROTECTED_DOT]", ".") for line in lines_protected]
    return lines

In [101]:
x=split_into_sentences(s)# form array of sentences in one news 
    

In [102]:
all_sent=[]
for i in df:
    preprocessed=preprocess(i)
    sentence_list=split_into_sentences(preprocessed)
    all_sent=all_sent+sentence_list
    

In [105]:
arr = np.array(all_sent)
arr.shape

(42441,)

In [106]:
import spacy
spacy.__version__

'3.1.1'

In [107]:
nlp=spacy.load("en_core_web_md")

In [108]:
import json
f1=open('Final_NameEntity.json', 'w') 

In [109]:
for i in all_sent:
    doc=nlp(i)
    entities = []
    labels = []
    position_start = []
    position_end = []
    for ent in doc.ents:
            entities.append(ent)
            labels.append(ent.label_)
            position_start.append(ent.start_char)
            position_end.append(ent.end_char)

    xf=[]
    for i in range (len(entities)):
            x=[]
            x.append(str(entities[i]))
            x.append(labels[i])
            xf.append(x)
    if len(xf)>0:
        json.dump(xf,f1)



In [111]:
f1.close()

In [25]:
xf=[]
for i in range (10):
    doc=nlp(all_sent[i])
    entities = []
    labels = []
    position_start = []
    position_end = []
    for ent in doc.ents:

        entities.append(ent)
        labels.append(ent.label_)
        position_start.append(ent.start_char)
        position_end.append(ent.end_char)
    for i in range (len(entities)):
        x=[]
        x.append(str(entities[i]))
        x.append(labels[i])
        xf.append(x)

        for i in xf:
            #if i[1]=='ORG':
                
            print(i)

['Sri Lanka', 'GPE']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['post-Covid-19 Sri Lanka', 'ORG']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['post-Covid-19 Sri Lanka', 'ORG']
['The Doha Round of Multilateral Trade Negotiations', 'EVENT']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['post-Covid-19 Sri Lanka', 'ORG']
['The Doha Round of Multilateral Trade Negotiations', 'EVENT']
['US', 'GPE']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['post-Covid-19 Sri Lanka', 'ORG']
['The Doha Round of Multilateral Trade Negotiations', 'EVENT']
['US', 'GPE']
['WTO', 'ORG']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG']
['post-Covid-19 Sri Lanka', 'ORG']
['The Doha Round of Multilateral Trade Negotiations', 'EVENT']
['US', 'GPE']
['WTO', 'ORG']
['Sri Lanka', 'GPE']
['Sri Lanka', 'GPE']
['Pathfinder Foundation Study Group', 'ORG

In [ ]:
######starts IE and triples stufff

In [207]:
#####
def extract_relations(doc):

    with doc.retokenize() as retokenizer:
        
        for ent in doc.ents:
            if ent.label_ in ["ORG"]:
                retokenizer.merge(ent)
    
    triples = []
    for ent in doc.ents:
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "prep"]
        for prep in preps:
            for child in prep.children:
                triples.append((ent.text, "{} {}".format(ent.root.head, prep), child.text))
            
    
    return triples

In [215]:
def extract_relations(doc):
    
    triples = []
    with doc.retokenize() as retokenizer: 
        for ent in doc.ents:
            if ent.label_ in ["ORG"]:
                retokenizer.merge(ent)
    print(doc.ents)
    for ent in doc.ents:
        print(ent)
        print('asssss')
        print(ent.root.head)
        preps = [prep for prep in ent.root.head.children if prep.dep_ == "nsubj"]
        print(preps)
        for prep in preps:
            for child in prep.children:
                triples.append((ent.text, "{} {}".format(ent.root.head, prep), child.text))
            
    
    return triples

In [216]:
p=extract_relations(nlp(t))

(amzon, microsoft, Apple)
amzon
asssss
bought
[amzon]
microsoft
asssss
bought
[amzon]
Apple
asssss
to
[]


In [201]:
t="amzon bought microsoft including all its staffs and then sold it to the Apple"

In [203]:
import spacy
from spacy import displacy

doc = nlp(t)
displacy.serve(doc, style="dep")

/Users/sivapalansarangan/opt/anaconda3/lib/python3.7/site-packages/spacy/displacy/__init__.py:97: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [208]:
p=extract_relations(nlp(t))

(amzon, microsoft, Apple)


In [198]:
 for i in x:
        print("\n" + i)
        doc=nlp(i)
        relations = extract_relations(doc)
        print(relations)
        


  Asia Asset Finance PLC (AAF)  has gained stable outlook with resilience overcoming growth headwinds in Covid-19 environment, leading Credit Rating Agency ICRA Lanka Limited affirmed.
(Asia Asset Finance PLC, AAF, Covid-19 environment, Credit Rating Agency, ICRA Lanka Limited)
[]

 It has revised the outlook of Asia Asset Finance PLC to BBB+[Stable] from BBB+(Negative)  considering growth performance with capital, operational, financial and managerial support from its parent company Muthoot Finance Limited (MFL) .
(Asia Asset Finance PLC, Muthoot Finance Limited, MFL)
[]

 The outlook revision to “Stable” from “Negative” reflects the improvement of the parent entity’s rating and the additional support extended by the parent, as well as the increase in exposure to asset backed lending, including Gold Loans.
(Gold Loans,)
[]

 Further, ICRA Lanka noted that the capital adequacy ratios of AAF remain an adequate buffer over the minimum capital requirement (set by the CBSL)  continues mod

In [ ]:
####ends here the IE

In [129]:
import csv
f = open('samp.csv', 'w')

# create the csv writer
writer = csv.writer(f)

header=["Sentence","Organizations"]# write a row to the csv file
writer.writerow(header)

# close the file

24

In [130]:
for i in all_sent:
    doc=nlp(i)
    for ent in doc.ents:
        if ent.label_=='ORG':
            l=[i,str(ent)]
            writer.writerow(l)
    
    
    

In [131]:
f.close()